In [1]:
import os
import pprint

# Define the target directory
target_directory = r"C:\Users\pablosal\Desktop\gbbai-agent-architecture-lab"  # change your directory here

# Check if the directory exists
if os.path.exists(target_directory):
    # Change the current working directory
    os.chdir(target_directory)
    print(f"Directory changed to {os.getcwd()}")
else:
    print(f"Directory {target_directory} does not exist.")

Directory changed to C:\Users\pablosal\Desktop\gbbai-agent-architecture-lab


### Understanding Semantic Kernel Components

Before diving into the implementation, let's understand the key components of Semantic Kernel.

#### The Kernel

The Kernel is the central orchestrator in Semantic Kernel. It manages:

- **Connecting to AI Models**: Interfaces with various AI models via connectors.
- **Registering and Invoking Plugins**: Manages the lifecycle and execution of plugins.
- **Managing Memory and Context**: Maintains state and context across interactions.
- **Interacting with the Planner**: Coordinates with the Planner to achieve specified goals.

#### Connectors

Connectors allow the Kernel to interface with various AI models and services. They define how the Kernel communicates with these models, whether they're:

- **OpenAI Models**: Such as GPT-3.5, GPT-4.
- **Azure OpenAI Services**: Leveraging Microsoft's cloud-based AI capabilities.
- **Local Models**: Using libraries like Hugging Face Transformers.

#### Plugins

Plugins are modular units that extend the Kernel's functionality. They consist of:

- **Prompt Functions**: Use natural language prompts to interact with AI models.
- **Native Functions**: Written in Python, performing deterministic tasks or interfacing with external services.

Plugins act as building blocks for complex workflows.

#### The Planner

The Planner uses AI to dynamically create a sequence of actions (a plan) to achieve a specified goal. It considers:

- **Available Plugins and Their Functions**: Understands the capabilities of each plugin.
- **Function Descriptions**: Uses metadata to understand what each function does.
- **Combining Functions**: Determines how functions can be combined to fulfill the goal.


### 🌟 Creating a Multi-Agent Architecture for Medical Research Documentation

In the realm of medical research, producing high-quality documentation is crucial. This guide aims to design a multi-agent system consisting of three specialized agents:

- **Medical Researcher**: Gathers and summarizes relevant medical information.
- **Clinical Evaluator**: Assesses the clinical relevance and accuracy of the information.
- **Medical Editor**: Refines the language and ensures adherence to medical writing standards.

Using Semantic Kernel's capabilities, we'll create plugins representing these agents and use a Planner to dynamically orchestrate their workflow based on a given goal.

By leveraging Semantic Kernel's capabilities, we can create a robust single-agent and sophisticated multi-agent system. Let's do it !

### Implementation

We will:

1. **Define Plugins**: Create plugins for the Medical Researcher, Clinical Evaluator, and Medical Editor.
2. **Configure the Kernel**: Set up the Kernel with the necessary connectors and plugins.
3. **Develop the Planner**: Implement the Planner to dynamically orchestrate the workflow.
4. **Execute the Plan**: Run the system to produce high-quality medical documentation.

In [2]:
#!pip install semantic-kernel==0.9.3b1
print(f"We are using the following semantic_kernel library version: 0.9.3b1")

We are using the following semantic_kernel library version: 0.9.3b1


## 1. Define Plugins

We have defined the plugin for MedicalAgents located in the specified directory  `C:\Users\pablosal\Desktop\gbbai-agent-architecture-lab\src\plugins\MedicalAgents`

## 2. Configure The Kernel

In [3]:
import semantic_kernel as sk

# Initialize the kernel
kernel = sk.Kernel()

In [4]:
# add Azure OpenAI service connector to the kernel

import os
from semantic_kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion

# Load environment variables from a .env file
from dotenv import load_dotenv
load_dotenv()

AZURE_OPENAI_KEY = os.getenv("AZURE_OPENAI_KEY")
AZURE_AOAI_CHAT_MODEL_NAME_DEPLOYMENT_ID = os.getenv("AZURE_AOAI_CHAT_MODEL_NAME_DEPLOYMENT_ID")
AZURE_OPENAI_API_ENDPOINT = os.getenv("AZURE_OPENAI_API_ENDPOINT")
AZURE_OPENAI_API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION")

openai_service = AzureChatCompletion(deployment_name=AZURE_AOAI_CHAT_MODEL_NAME_DEPLOYMENT_ID,
                                    endpoint=AZURE_OPENAI_API_ENDPOINT, 
                                    service_id="openai_chat", 
                                    api_key=AZURE_OPENAI_KEY)

# Create a Kernel instance and add the OpenAI service
kernel = Kernel()
kernel.add_service(openai_service)

In [85]:
from semantic_kernel.connectors.ai.open_ai import OpenAIChatPromptExecutionSettings

execution_config = OpenAIChatPromptExecutionSettings(service_id = "openai_chat",
                                                    max_tokens=100,
                                                    temperature=0,
                                                    top_p=0.0)

execution_settings = OpenAIChatPromptExecutionSettings(
    service_id="openai_chat",
    max_tokens=500,
    temperature=0.7,
    top_p=0.9,
    seed=42
)

prompt = """
You are a medical researcher. Summarize the latest research articles on the topic of {{$input}}. 
Include key findings, methodologies, and any significant conclusions.
"""

generate_medical_summary = kernel.create_function_from_prompt(
    prompt=prompt,
    plugin_name="Generate_Medical_Summary",
    function_name="generate_summary",
    execution_settings=execution_settings
)

response = await kernel.invoke(generate_medical_summary, input="heart disease treatments")

print("Generated Medical Research Summary:", response)

Generated Medical Research Summary: As of the latest research articles up to October 2023, several key findings and advancements have been made in the treatment of heart disease. Here is a summary of the most significant studies:

1. **Gene Therapy for Heart Failure**:
   - **Key Findings**: A study published in *Nature Medicine* demonstrated the potential of gene therapy to improve heart function in patients with heart failure. Researchers used a viral vector to deliver a gene encoding for a protein that enhances calcium handling in heart cells.
   - **Methodology**: The study involved a randomized controlled trial with 150 participants who received either the gene therapy or a placebo. Cardiac function was assessed using echocardiography and MRI.
   - **Conclusions**: Patients receiving the gene therapy showed significant improvements in ejection fraction and overall heart function compared to the placebo group, suggesting a promising new avenue for heart failure treatment.

2. **Nov

In [5]:
from semantic_kernel.planners.basic_planner import BasicPlanner
from semantic_kernel.planners.sequential_planner import SequentialPlanner


plugin_directory = r"C:\Users\pablosal\Desktop\gbbai-agent-architecture-lab\src\plugins"
medical_agents_plugin = kernel.import_plugin_from_prompt_directory(plugin_directory, "MedicalAgents")

In [6]:
print("Available functions in MedicalAgents plugin:")
medical_agents_plugin.functions


Available functions in MedicalAgents plugin:


{'ClinicalEvaluator': KernelFunctionFromPrompt(metadata=KernelFunctionMetadata(name='ClinicalEvaluator', plugin_name='MedicalAgents', description='Evaluates the clinical relevance and accuracy of a medical summary.', parameters=[KernelParameterMetadata(name='input', description='The medical summary to be evaluated.', default_value='', type_='', is_required=True, type_object=None)], is_prompt=True, is_asynchronous=True, return_parameter=KernelParameterMetadata(name='return', description='The completion result', default_value=None, type_='FunctionResult', is_required=True, type_object=None)), prompt_template=KernelPromptTemplate(prompt_template_config=PromptTemplateConfig(name='ClinicalEvaluator', description='Evaluates the clinical relevance and accuracy of a medical summary.', template='You are a Clinical Evaluator. Review the following medical summary for clinical relevance and accuracy:\n\nMedical Summary:\n{{$input}}\n\nProvide an evaluation highlighting any inaccuracies or areas ne

## 3. Develop the Planer

In [7]:
# Initialize the planner
planner = BasicPlanner('openai_chat')

# Define the goal
goal_prompt = """
I want to produce a high-quality medical document on the topic of 'The impact of AI on medical diagnostics.'
"""

# Create the plan
plan = await planner.create_plan(goal_prompt, kernel)

In [8]:
print(plan.generated_plan)

{
    "input": "The impact of AI on medical diagnostics",
    "subtasks": [
        {"function": "MedicalAgents.MedicalResearcher", "args": {"input": "The impact of AI on medical diagnostics"}},
        {"function": "MedicalAgents.MedicalEditor"}
    ]
}


## 4. Execute the Plan

In [9]:
poem = await planner.execute_plan(plan, kernel)


In [10]:
print(poem)

The impact of artificial intelligence (AI) on medical diagnostics is a rapidly evolving field, with numerous studies highlighting its potential to enhance diagnostic accuracy, efficiency, and accessibility. Below is a summary of key findings from recent research:

1. **Diagnostic Accuracy and Efficiency**:
   - A study published in *The Lancet Digital Health* (2022) demonstrated that AI algorithms could match or exceed the diagnostic accuracy of human experts in detecting diseases such as diabetic retinopathy, lung cancer, and melanoma from medical images. For instance, an AI system achieved an accuracy of 94.5% in diagnosing diabetic retinopathy compared to 91.3% by ophthalmologists.
   - Research in *Nature Medicine* (2022) revealed that an AI model outperformed radiologists in detecting breast cancer from mammograms, reducing false positives by 5.7% and false negatives by 9.4%.

2. **AI in Radiology**:
   - A comprehensive review in the *Journal of the American College of Radiology*